In [2]:
import sqlite3
import UnityPy
from UnityPy.enums import ClassIDType

timeline_path = "storytimeline_041097001"
storyline_name = "storytimeline_041097001"

env = UnityPy.load(timeline_path)

In [30]:
text_objects = []
for obj in env.objects:
    obj_read = obj.read()
    if hasattr(obj_read, 'Text'):
        text_objects.append(obj_read)

In [37]:
target_columns = ['Name','Text','VoiceSheetId','CharaId','CueId']

In [50]:
num_blocks = len(text_objects)
# last block has NextBlock of -1
# first block has NextBlock of 1
last_block_block_num = max([x.NextBlock for x in text_objects])

In [205]:
t_processed_dict = {}
for t in text_objects:
    t_processed = {}
    t_processed['RubyInfo'] = []
    if t.NextBlock == -1:
        t_processed['BlockNumber'] = last_block_block_num
    else:
        t_processed['BlockNumber'] = t.NextBlock - 1
    for c in target_columns:
        t_processed[c] = getattr(t, c)
    t_processed_dict[t_processed['BlockNumber']] = t_processed

In [206]:
t_processed_list = []
for i in range(num_blocks):
    t_processed_list.append(t_processed_dict[i])

In [207]:
ruby_path = "ast_ruby_041097001"
ruby_name = "ast_ruby_041097001"

In [208]:
env_ruby = UnityPy.load(ruby_path)

In [209]:
ruby_obj = None
for obj in env_ruby.objects:
    if obj.read().m_Name == ruby_name:
        print("Ruby found")
        ruby_obj = obj.read()
        break

Ruby found


In [210]:
ruby_object_columns = ['CharX','CharY','RubyText']

In [211]:
for da in ruby_obj.DataArray:
    ruby_info_list = []
    for rbl in da.RubyDataList:
        ruby_info_obj = {}
        for roc in ruby_object_columns:
            ruby_info_obj[roc] = getattr(rbl, roc)
        ruby_info_list.append(ruby_info_obj)
    t_processed_list[da.BlockIndex]['RubyInfo'] = ruby_info_list

In [212]:
t_processed_list

[{'RubyInfo': [],
  'BlockNumber': 0,
  'Name': '',
  'Text': '',
  'VoiceSheetId': '',
  'CharaId': -1,
  'CueId': -1},
 {'RubyInfo': [],
  'BlockNumber': 1,
  'Name': '',
  'Text': '――――――――',
  'VoiceSheetId': '041097001',
  'CharaId': 1097,
  'CueId': 0},
 {'RubyInfo': [],
  'BlockNumber': 2,
  'Name': '',
  'Text': '混濁する意識の中で、いつも……いつも、同じ問いが浮かぶ。\n“もしあの時、この結末を知っていたら……どうしただろうか？”',
  'VoiceSheetId': '',
  'CharaId': 90001,
  'CueId': -1},
 {'RubyInfo': [],
  'BlockNumber': 3,
  'Name': '',
  'Text': '…………考える。それが徒労に終わることを知りながら。\nなぜなら……答えもまた、いつも同じだからだ。',
  'VoiceSheetId': '',
  'CharaId': 90001,
  'CueId': -1},
 {'RubyInfo': [],
  'BlockNumber': 4,
  'Name': '',
  'Text': '――――――――',
  'VoiceSheetId': '041097001',
  'CharaId': 1097,
  'CueId': 1},
 {'RubyInfo': [],
  'BlockNumber': 5,
  'Name': '',
  'Text': 'そうして再び……意識が遠のいていく。全てが曖昧になる中でも……\nはっきりと、感じる。逃れがたいそれが、今も……まだ……――',
  'VoiceSheetId': '',
  'CharaId': 90001,
  'CueId': -1},
 {'RubyInfo': [],
  'BlockNumber': 6,
  'Name': '',
  'Te

In [105]:
import acb

In [188]:
acb_base_path = "C:\\Users\\Matt\\Desktop\\uma-voice-dataset-creator\\snd_voi_story_041097001"
acb_path = acb_base_path + ".acb"
awb_path = acb_base_path + ".awb"
uma_hca_key = "0x450D608C479F"
vgastream_cli_path = "C:\\Users\\Matt\\Desktop\\uma-voice-dataset-creator\\vgmstream\\vgmstream-cli.exe"

In [189]:
acbfile = acb.ACBFile(acb_path, awb_path, hca_keys = uma_hca_key)

In [199]:
save_dir = "C:\\Users\\Matt\\Desktop\\uma-voice-dataset-creator\\output"

In [202]:
import os
import subprocess
def save_track(acb_file, track, target_dir, name):
    os.makedirs(target_dir, exist_ok=True)
    name_hca = name + ".hca"
    name_wav = name + ".wav"
    with open(os.path.join(target_dir, name_hca), "wb") as out_file:
        out_file.write(acb_file.get_track_data(track, True))
    subprocess.run([vgastream_cli_path, '-o', os.path.join(target_dir, name_wav), os.path.join(target_dir, name_hca)])
    os.remove(os.path.join(target_dir, name_hca))

In [183]:
acbfile.track_list.tracks

[track_t(cue_id=0, name='snd_voi_story_041097001_00', memory_wav_id=65535, external_wav_id=0, enc_type=2, is_stream=1),
 track_t(cue_id=1, name='snd_voi_story_041097001_01', memory_wav_id=65535, external_wav_id=1, enc_type=2, is_stream=1),
 track_t(cue_id=2, name='snd_voi_story_041097001_02', memory_wav_id=65535, external_wav_id=2, enc_type=2, is_stream=1),
 track_t(cue_id=3, name='snd_voi_story_041097001_03', memory_wav_id=65535, external_wav_id=3, enc_type=2, is_stream=1),
 track_t(cue_id=4, name='snd_voi_story_041097001_04', memory_wav_id=65535, external_wav_id=4, enc_type=2, is_stream=1),
 track_t(cue_id=5, name='snd_voi_story_041097001_05', memory_wav_id=65535, external_wav_id=5, enc_type=2, is_stream=1),
 track_t(cue_id=6, name='snd_voi_story_041097001_06', memory_wav_id=65535, external_wav_id=6, enc_type=2, is_stream=1),
 track_t(cue_id=7, name='snd_voi_story_041097001_07', memory_wav_id=65535, external_wav_id=7, enc_type=2, is_stream=1),
 track_t(cue_id=8, name='snd_voi_story_0

In [185]:
acbfile.track_list.tracks[0].name

'snd_voi_story_041097001_00'

In [203]:
for track in acbfile.track_list.tracks:
    save_track(acbfile, track, save_dir, track.name)

In [215]:
t_processed_list[1]['CueId']

0

In [220]:
# invariant: cue_id = index for tracks
for t in t_processed_list:
    if t['CueId'] == -1:
        continue
    curr_track = acbfile.track_list.tracks[t['CueId']]
    save_track(acbfile, curr_track, save_dir, curr_track.name)
    t["AudioFileLocation"] = os.path.join(save_dir, curr_track.name) + ".wav"
    
    

In [221]:
t_processed_list

[{'RubyInfo': [],
  'BlockNumber': 0,
  'Name': '',
  'Text': '',
  'VoiceSheetId': '',
  'CharaId': -1,
  'CueId': -1},
 {'RubyInfo': [],
  'BlockNumber': 1,
  'Name': '',
  'Text': '――――――――',
  'VoiceSheetId': '041097001',
  'CharaId': 1097,
  'CueId': 0,
  'AudioFileLocation': 'C:\\Users\\Matt\\Desktop\\uma-voice-dataset-creator\\output\\snd_voi_story_041097001_00.wav'},
 {'RubyInfo': [],
  'BlockNumber': 2,
  'Name': '',
  'Text': '混濁する意識の中で、いつも……いつも、同じ問いが浮かぶ。\n“もしあの時、この結末を知っていたら……どうしただろうか？”',
  'VoiceSheetId': '',
  'CharaId': 90001,
  'CueId': -1},
 {'RubyInfo': [],
  'BlockNumber': 3,
  'Name': '',
  'Text': '…………考える。それが徒労に終わることを知りながら。\nなぜなら……答えもまた、いつも同じだからだ。',
  'VoiceSheetId': '',
  'CharaId': 90001,
  'CueId': -1},
 {'RubyInfo': [],
  'BlockNumber': 4,
  'Name': '',
  'Text': '――――――――',
  'VoiceSheetId': '041097001',
  'CharaId': 1097,
  'CueId': 1,
  'AudioFileLocation': 'C:\\Users\\Matt\\Desktop\\uma-voice-dataset-creator\\output\\snd_voi_story_041097001_01.wav'},
 {'Ruby

In [223]:
meta_file_path = "C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\meta"
master_file_path = "C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\master\\master.mdb"

In [225]:
import sqlite3
meta_con = sqlite3.connect(meta_file_path)
master_con = sqlite3.connect(master_file_path)
meta_cur = meta_con.cursor()
master_cur = master_con.cursor()

In [232]:
story_ids = master_cur.execute("SELECT story_id FROM chara_story_data WHERE chara_id = 1097").fetchall()

In [234]:
story_ids = [x[0] for x in story_ids]

In [235]:
story_ids

[41097001, 41097002, 41097003, 41097004, 41097005, 41097006, 41097007]

In [267]:
def get_hash_given_name_filter(name_filter, meta_cursor):
    res = meta_cursor.execute(f"SELECT n, h FROM a WHERE n LIKE '%{name_filter}%'").fetchall()
    return res

In [304]:
file_packets = []
for story_id in story_ids:
    story_id_str = str(story_id)
    res = get_hash_given_name_filter(f"%{story_id_str}", meta_cur)
    curr_list = []
    for r in res:
        if r[0][0] != 's' or 'resource' in r[0]:
            continue
        curr_item = {}
        curr_item['name'] = r[0]
        curr_item['hash'] = r[1]
        curr_list.append(curr_item)
    #res = [r for r in res]
    #res = list(filter(lambda s: s[0] == 's' and 'resource' not in s and 'dot' not in s, res))
    file_packets.append(curr_list)

In [306]:
file_packet

[{'name': 'story/data/04/1097/ast_dot_041097001',
  'hash': 'IYPY46AWJXDTBACX6MZGUZ3LV27OTOZM'},
 {'name': 'story/data/04/1097/ast_ruby_041097001',
  'hash': 'PA2WM2BDNQ6Y26SHRJPW3QFLBZ44GFNV'},
 {'name': 'story/data/04/1097/storytimeline_041097001',
  'hash': 'XCTZGQRIBGK24V5CTWR2F52VBNIAFFCT'},
 {'name': 'sound/c/snd_voi_story_041097001.acb',
  'hash': 'UIP3TZ34FSPQQKZIOUAGXENLNAOT5O2F'},
 {'name': 'sound/c/snd_voi_story_041097001.awb',
  'hash': 'BIUHGZRF5SNDUHSMX4BRU7SHDGF7RTZ3'}]

In [290]:
hash_files_base_path = "C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\dat"

In [292]:
curr_hash = file_packet[0]['hash']

In [293]:
curr_hash

'IYPY46AWJXDTBACX6MZGUZ3LV27OTOZM'

In [296]:
def get_filepath_from_hash(h, base_path):
    return os.path.join(base_path, h[:2], h)

In [297]:
get_filepath_from_hash(curr_hash, hash_files_base_path)

'C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\dat\\IY\\IYPY46AWJXDTBACX6MZGUZ3LV27OTOZM'

In [310]:
for file_packet in file_packets:
    for item in file_packet:
        item["path"] = get_filepath_from_hash(item["hash"], hash_files_base_path)

In [312]:
file_packet = file_packets[0]

In [313]:
file_packet

[{'name': 'story/data/04/1097/ast_dot_041097001',
  'hash': 'IYPY46AWJXDTBACX6MZGUZ3LV27OTOZM',
  'path': 'C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\dat\\IY\\IYPY46AWJXDTBACX6MZGUZ3LV27OTOZM'},
 {'name': 'story/data/04/1097/ast_ruby_041097001',
  'hash': 'PA2WM2BDNQ6Y26SHRJPW3QFLBZ44GFNV',
  'path': 'C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\dat\\PA\\PA2WM2BDNQ6Y26SHRJPW3QFLBZ44GFNV'},
 {'name': 'story/data/04/1097/storytimeline_041097001',
  'hash': 'XCTZGQRIBGK24V5CTWR2F52VBNIAFFCT',
  'path': 'C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\dat\\XC\\XCTZGQRIBGK24V5CTWR2F52VBNIAFFCT'},
 {'name': 'sound/c/snd_voi_story_041097001.acb',
  'hash': 'UIP3TZ34FSPQQKZIOUAGXENLNAOT5O2F',
  'path': 'C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\dat\\UI\\UIP3TZ34FSPQQKZIOUAGXENLNAOT5O2F'},
 {'name': 'sound/c/snd_voi_story_041097001.awb',
  'hash': 'BIUHGZRF5SNDUHSMX4BRU7SHDGF7RTZ3',
  'path': 'C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\

In [6]:
import sqlite3
import UnityPy
from UnityPy.enums import ClassIDType
import acb
import os
import subprocess
import csv, json
from pathlib import Path
import torch
from transformers import pipeline

target_columns = ['Name','Text','VoiceSheetId','CharaId','CueId']
ruby_object_columns = ['CharX','CharY','RubyText']
uma_hca_key = "0x450D608C479F"

meta_file_path = "C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\meta"
master_file_path = "C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\master\\master.mdb"
hash_files_base_path = "C:\\Users\\Matt\\AppData\\LocalLow\\Cygames\\umamusume\\dat"

vgastream_cli_path = "C:\\Users\\Matt\\Desktop\\uma-voice-dataset-creator\\vgmstream\\vgmstream-cli.exe"

save_dir_base_path = "C:\\Users\\Matt\\Desktop\\uma-voice-dataset-creator\\output"

generate_kwargs = {
    "language": "Japanese",
    "no_repeat_ngram_size": 0,
    "repetition_penalty": 1.0,
}
pipe = pipeline(
    "automatic-speech-recognition",
    model="litagin/anime-whisper",
    device="cuda",
    torch_dtype=torch.float16,
    chunk_length_s=30.0,
    batch_size=64,
)

def get_file_packets(chara_id: int):
    def get_hash_given_name_filter(name_filter, meta_cursor):
        res = meta_cursor.execute(f"SELECT n, h FROM a WHERE n LIKE '%{name_filter}%'").fetchall()
        return res
    def get_filepath_from_hash(h, base_path):
        return os.path.join(base_path, h[:2], h)
    meta_con = sqlite3.connect(meta_file_path)
    master_con = sqlite3.connect(master_file_path)
    meta_cur = meta_con.cursor()
    master_cur = master_con.cursor()
    story_ids = master_cur.execute(f"SELECT story_id FROM chara_story_data WHERE chara_id = {str(chara_id)}").fetchall()
    story_ids = [x[0] for x in story_ids]
    file_packets = []
    for story_id in story_ids:
        story_id_str = str(story_id)
        res = get_hash_given_name_filter(f"%{story_id_str}", meta_cur)
        curr_list = []
        for r in res:
            if r[0][0] != 's' or 'resource' in r[0]:
                continue
            curr_item = {}
            curr_item['name'] = r[0]
            curr_item['hash'] = r[1]
            curr_list.append(curr_item)
        #res = [r for r in res]
        #res = list(filter(lambda s: s[0] == 's' and 'resource' not in s and 'dot' not in s, res))
        file_packets.append(curr_list)
    for file_packet in file_packets:
        for item in file_packet:
            item["path"] = get_filepath_from_hash(item["hash"], hash_files_base_path)
    return file_packets
    
def save_track(acb_file, track, target_dir, name):
    os.makedirs(target_dir, exist_ok=True)
    name_hca = name + ".hca"
    name_wav = name + ".wav"
    with open(os.path.join(target_dir, name_hca), "wb") as out_file:
        out_file.write(acb_file.get_track_data(track, True))
    subprocess.run([vgastream_cli_path, '-o', os.path.join(target_dir, name_wav), os.path.join(target_dir, name_hca)])
    os.remove(os.path.join(target_dir, name_hca))

def process_file_packet(file_packet, chara_id: int):
    # handle timeline
    timeline_item = list(filter(lambda x: "timeline" in x["name"], file_packet))
    assert len(timeline_item) == 1
    timeline_item = timeline_item[0]
    #print(timeline_item)
    timeline_path = timeline_item["path"]
    storyline_name = timeline_item["name"]
    
    last_slash_index = storyline_name.rfind("/")
    storyline_name = storyline_name[last_slash_index+1:]

    env = UnityPy.load(timeline_path)
    text_objects = []
    for obj in env.objects:
        obj_read = obj.read()
        if hasattr(obj_read, 'Text'):
            text_objects.append(obj_read)
    num_blocks = len(text_objects)
    # last block has NextBlock of -1
    # first block has NextBlock of 1
    last_block_block_num = max([x.NextBlock for x in text_objects])
    t_processed_dict = {}
    for t in text_objects:
        t_processed = {}
        t_processed['RubyInfo'] = []
        if t.NextBlock == -1:
            t_processed['BlockNumber'] = last_block_block_num
        else:
            t_processed['BlockNumber'] = t.NextBlock - 1
        for c in target_columns:
            t_processed[c] = getattr(t, c)
        t_processed_dict[t_processed['BlockNumber']] = t_processed
    t_processed_list = []
    for i in range(num_blocks):
        t_processed_list.append(t_processed_dict[i])

    # handle ruby
    ruby_item = list(filter(lambda x: "ruby" in x["name"], file_packet))
    assert len(ruby_item) == 1
    ruby_item = ruby_item[0]
    #print(ruby_item)
    ruby_path = ruby_item["path"]
    ruby_name = ruby_item["name"]
    ruby_name = ruby_name[ruby_name.rfind("/")+1:]
    env_ruby = UnityPy.load(ruby_path)
    #print(t_processed_list)
    ruby_obj = None
    for obj in env_ruby.objects:
        if obj.read().m_Name == ruby_name:
            #print("Ruby found")
            ruby_obj = obj.read()
            break
    if ruby_obj:
        for da in ruby_obj.DataArray:
            ruby_info_list = []
            for rbl in da.RubyDataList:
                ruby_info_obj = {}
                for roc in ruby_object_columns:
                    ruby_info_obj[roc] = getattr(rbl, roc)
                ruby_info_list.append(ruby_info_obj)
            #print(da.BlockIndex)
            #print(ruby_info_list)
            t_processed_list[da.BlockIndex]['RubyInfo'] = ruby_info_list
    #print(t_processed_list)

    # handle voices
    acb_item = list(filter(lambda x: ".acb" in x["name"], file_packet))[0]
    awb_item = list(filter(lambda x: ".awb" in x["name"], file_packet))[0]
    acb_path = acb_item["path"]
    awb_path = awb_item["path"]
    acbfile = acb.ACBFile(acb_path, awb_path, hca_keys = uma_hca_key)
    save_dir = save_dir_base_path

    for track in acbfile.track_list.tracks:
        save_track(acbfile, track, save_dir, track.name)

    # invariant: cue_id = index for tracks
    for t in t_processed_list:
        if t['CueId'] == -1:
            continue
        curr_track = acbfile.track_list.tracks[t['CueId']]
        save_track(acbfile, curr_track, save_dir, curr_track.name)
        t["AudioFileLocation"] = os.path.join(save_dir, curr_track.name) + ".wav"
        audio_path = t["AudioFileLocation"]
        result = pipe(audio_path, generate_kwargs=generate_kwargs)
        t["TextAnimeWhisper"] = result["text"]
    print(f"{storyline_name} done")
    return t_processed_list

def write_simple(data, out):
    keys = sorted({k for d in data for k in d})
    with Path(out).open('w', encoding='utf-8', newline='') as f:
        w = csv.DictWriter(f, fieldnames=keys, quoting=csv.QUOTE_ALL)
        w.writeheader()
        for d in data:
            w.writerow({k: json.dumps(v, ensure_ascii=False) if isinstance(v, (list, dict)) else v
                        for k, v in d.items()})

def process_for_char(chara_id: int):
    file_packets = get_file_packets(chara_id)
    #print(file_packets)
    final_list = []
    for file_packet in file_packets:
        final_list.extend(process_file_packet(file_packet, chara_id))
    write_simple(final_list, os.path.join(save_dir_base_path, "info.csv"))
    return final_list



Device set to use cuda
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [7]:
final_list = process_for_char(1097)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


storytimeline_041097001 done
storytimeline_041097002 done
storytimeline_041097003 done
storytimeline_041097004 done
storytimeline_041097005 done
storytimeline_041097006 done
storytimeline_041097007 done


In [9]:
# Requires transformers>=4.51.0
# Requires sentence-transformers>=2.7.0

from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer("Qwen/Qwen3-Embedding-4B")

# We recommend enabling flash_attention_2 for better acceleration and memory saving,
# together with setting `padding_side` to "left":
# model = SentenceTransformer(
#     "Qwen/Qwen3-Embedding-4B",
#     model_kwargs={"attn_implementation": "flash_attention_2", "device_map": "auto"},
#     tokenizer_kwargs={"padding_side": "left"},
# )

# The queries and documents to embed
queries = [
    "What is the capital of China?",
    "Explain gravity",
]
documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun.",
]

# Encode the queries and documents. Note that queries benefit from using a prompt
# Here we use the prompt called "query" stored under `model.prompts`, but you can
# also pass your own prompt via the `prompt` argument
query_embeddings = model.encode(queries, prompt_name="query")
document_embeddings = model.encode(documents)

# Compute the (cosine) similarity between the query and document embeddings
similarity = model.similarity(query_embeddings, document_embeddings)
print(similarity)
# tensor([[0.7534, 0.1147],
#         [0.0320, 0.6258]])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Matt\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Matt\.cache\huggingface\hub\models--Qwen--Qwen3-Embedding-4B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OSError: The paging file is too small for this operation to complete. (os error 1455)